# Collecting the data
First we have to get all the needed data. We will use web3py with Pancake bet contract (0x0E3A8078EDD2021dadcdE733C6b4a86E51EE8f07) and requests with bscscan API.

Add needed modules folders:

In [ ]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

if module_path + '/src' not in sys.path:
    sys.path.append(module_path + '/src')

# os.chdir("..")

print(os.getcwd())

## Getting active players
Use get_active_players.py main() with start and end time. It may be required to change the main function loop to iterate through more past transactions.
Also change CURRENT_BLOCK_NUMBER to the current one - to start checking transactions before this block.

In [ ]:
import datetime

import get_active_players

# The same timeframe as for the training set
time_from_active_players = int(datetime.datetime(2022, 10, 28, 0, 0).timestamp())
time_to_active_players = int(datetime.datetime(2022, 12, 28, 0, 0).timestamp())
active_players_file = f'../data/active_players_2months.csv'

get_active_players.make_active_players_file(time_from_active_players, time_to_active_players, 70, active_players_file)

List of active players and their occurence within given timeframe:

In [ ]:
import pandas as pd
active_players_df = pd.read_csv(active_players_file, index_col=False)

active_players_df

Let's select only players that have made more than 100 bets in the selected timeframe.

In [ ]:
# List of players to download data of (you can get them from the "Getting active players" section or from the Pancake Prediction leaderboard)
main_wallets_list = active_players_df.loc[active_players_df['count'] > 100, 'player'].tolist()
print(f"Selected {len(main_wallets_list)} wallets")


## Downloading and preparing the data

### All needed variables

In [ ]:
# Timeframe to do all the computing (Training + Validation + Test sets)
time_from_get_bets = int(datetime.datetime(2022, 10, 28, 0, 0).timestamp())
time_to_get_bets = int(datetime.datetime(2023, 1, 28, 0, 0).timestamp())

  # to get all the transactions from bscscan api (depends on the time_from_get_bets)
block_from_get_bets = 22394098
block_to_get_bets = 25624791

# Downloading players bet history settings
players_data_folder = '../data/players_data/'

# Download rounds settings (check the rounds range according to the time_from_get_bets and time_to_get_bets values in the blockchain)
from_round = 36000
to_round = 65000
rounds_file = '../data/rounds_data/final_rounds_data.csv'

# Final data
final_data_folder = '../data/merged_data/'

### Downloading players bet history
We have to collect every needed player transaction data and save to separate JSON files.

In [ ]:
import download_players_bets
print(len(main_wallets_list))
download_players_bets.main_concurrent(main_wallets_list, players_data_folder, block_from_get_bets, block_to_get_bets)